# OpenPose to CSV generator
Author: David Pantoja

This script was written to crawl through each suitable video file in a folder, no matter how deep it was, run it using openpose, extract the 25 important joints' x&y coordinates, combine them with the rest of the file's coordinates, and produce a CSV. This process does open the video window, making your PC effectively unusable during generation time as the cursor refocus each time a new video is played.

Things to note:
    -This was only tested on Ubuntu and assumes openpose is functional on your system.
    -To stop the software, go to **Kernal** and press **Restart** or some similar variation of it, depending on what you want to accomplish. After that, click on the openpose windows that keep popping up and press **Esc**.

In [1]:
#Import relevant modules
import os
import pandas as pd
import numpy as np
import json
import re

# PLEASE EDIT THE BELOW CELL

In [2]:
#Location of openpose
openpose_dir = "/home/user/Code/realpose/openpose/"
#folder with videos in it, can contain folders inside that also have videos
data_folder = "/home/user/Desktop/videos/"
#location of output CSV
output_dir = "/home/user/Desktop/results/"

#Below are rarely changed
#Location of required .bin, this should rarely be changed, should NOT start with a dash
openpose_bin_path = "build/examples/openpose/openpose.bin"
#desired JSON location, this sets it to a folder in openpose. Automatically generated if not in folder
JSON_dir = openpose_dir + "tmp_output_JSON/"
folder_data_name = ""

In [3]:
#initializes empty dataframe with names columns
columns = ["NoseX","NeckX","RShoulderX","RElbowX","RWristX","LShoulderX","LElbowX","LWristX","MidHipX","RHipX","RKneeX","RAnkleX","LHipX","LKneeX","LAnkleX","REyeX","LEyeX","REarX","LEarX","LBigToeX","LSmallToeX","LHeelX","RBigToeX","RSmallToeX","RHeelX","NoseY","NeckY","RShoulderY","RElbowY","RWristY","LShoulderY","LElbowY","LWristY","MidHipY","RHipY","RKneeY","RAnkleY","LHipY","LKneeY","LAnkleY","REyeY","LEyeY","REarY","LEarY","LBigToeY","LSmallToeY","LHeelY","RBigToeY","RSmallToeY","RHeelY","File","FrameNo","Violation"]
#creates empty dataframe with columns above
df = pd.DataFrame(columns=columns)
#Sets working directory to what is given by openpose_dir
os.chdir(openpose_dir)
if not os.path.isdir(JSON_dir):
    os.mkdir(JSON_dir)

In [4]:
def set_folder_name():
    global folder_data_name
    path_split = data_folder.split("/", -1)
    path_split.reverse()
    for string in reversed(path_split):
        if string:
            folder_data_name = string

In [5]:
#properlly helps sort strings with numbers that indicate order
def atoi(text):
    return int(text) if text.isdigit() else text

#properlly sorts strings with numbers that indicate order
def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [6]:
#DELETES EVERY FILE IN THE TEMPORARY JSON FOLDER
def clear_JSON():
    for f in os.listdir(JSON_dir):
        if ".json" in f:
            os.remove(os.path.join(JSON_dir,f))

In [7]:
#adds JSON outputs to dataframe. Openpose purges the JSON folder after each video.
def add_video_to_df(dirpath, file):    
    #creates tmporary dataframe that will be appended to main later
    tmp_df = pd.DataFrame(columns=columns)
    
    #saves video name
    video_file = file
    
    #gets JSON file names in order
    JSON_files = os.listdir(JSON_dir)
    JSON_files.sort()
    
    #Counts frames
    frame = 0
    
    #Iterates through files in folder and adds the values to the dataframe
    for file in JSON_files:
        #makes sure it is a JSON file
        if ".json" in file:
        #gets json data
            with open(JSON_dir + file) as f:
                data = json.load(f)
        
            #Pulls relevant JSON data and appends it. JSON outline remains the same but this ONLY gets the values of the first person.
            arr = np.array(data["people"][0]["pose_keypoints_2d"])
            #deletes accuracy score
            arr = np.delete(arr, np.arange(2, arr.size, 3))
            #adds information that will go in PATH column
            arr = np.append(arr, dirpath + "/" + video_file)
            #adds information that will go in frame column
            arr = np.append(arr, frame)
        
            #adds information that will go in violation column, violation means the second folder's name in the branch
            violation = dirpath.replace(data_folder,"")
            violation = violation.split("/",1)
            violation = violation[0]
            violation = folder_data_name + "_" + violation
            arr = np.append(arr, violation)
        
            #reshapes data to fit dataframe
            tmp_row = pd.DataFrame(arr.reshape(-1, len(arr)), columns=columns)
        
            #adds frame dataframe information to video dataframe
            tmp_df = tmp_df.append(tmp_row, ignore_index=True)
        
            #increments video frame for only this video
            frame += 1
    #returns dataframe containing information of 1 video
    return tmp_df

In [ ]:
#Deltes JSON folder and runs one video through openpose to create new JSON data
def run_video(dirpath, file):
    if os.path.isdir(JSON_dir):
        clear_JSON()
    cmd = openpose_dir + openpose_bin_path + " --video " + dirpath +"/"+ file + " --write_json " + JSON_dir
    os.system(cmd)

In [ ]:
#for every branch in a directory do the following
set_folder_name()
for dirpath, dirnames, filenames in os.walk(data_folder):
    #sorts folders and files in order
    dirnames.sort(key=natural_keys)
    filenames.sort(key=natural_keys)
    for file in filenames:
        print("working on: " + dirpath + "/" + file)
        run_video(dirpath, file)
        df = df.append(add_video_to_df(dirpath, file), ignore_index=True)

In [ ]:
set_folder_name()
#exports dataframe containing information from all videos in data_folder as CSV to output_dir as "results.csv"
df.to_csv(output_dir + folder_data_name + "_results.csv", index=False, header=True)